<a href="https://colab.research.google.com/github/acmhp/MQTT_Temp_Umidade/blob/master/Zipf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import code
import theano.tensor as tt
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm

data = np.array( [1, 2, 27, 802, 2087, 18901, 14343, 21531] )

N = len( data )

print( "Number of data points: %d" % N )
def build_model():
    with pm.Model() as model:
        # unsure about the prior...
        #s = pm.Normal( 's', mu=0.0, sd=100 )
        #s = pm.HalfNormal( 's', sd=10 )

      s = pm.Gamma('s', alpha=1, beta=8)


      def logp( f ):
        r = tt.arange( 1, N+1 )
        return -s * tt.sum( f * tt.log(r) ) - tt.sum( f ) * tt.log( tt.sum(tt.power(1.0/r,s)) )

      pm.DensityDist( 'obs', logp=logp, observed={'f': data} )


      return model

def run( n_samples=10000 ):
    model = build_model()
    with model:
        start = pm.find_MAP()
        step = pm.NUTS( scaling=start )
        trace = pm.sample( n_samples, step=step, start=start )

    pm.summary( trace )
#    pm.traceplot( trace )
#    pm.plot_posterior( trace, kde_plot=True )
#    plt.show()

if __name__ == '__main__':
    run()

Number of data points: 8


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
Sequential sampling (2 chains in 1 job)
NUTS: [s]


Sampling 2 chains for 1_000 tune and 10_000 draw iterations (2_000 + 20_000 draws total) took 12 seconds.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
There were 35 divergences after tuning. Increase `target_accept` or reparameterize.
/usr/local/lib/python3.7/dist-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,
